<a href="https://colab.research.google.com/github/atedstone/unil_envi_ggl_hydrology_practicals/blob/main/supraglacial_q.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical: Interpreting supraglacial melt and discharge

In [ ]:
# Install ipympl so that we can use interactive plots
!pip install ipympl
# And activate in Colab
from google.colab import output
output.enable_custom_widget_manager()

**Important!** If this is the first time you are running the Notebook, restart your session now using the menu 'Runtime... Restart session'. You don't need to run the above cell again afterwards, you can continue with the rest of the Notebook.

In [ ]:
import requests
import os

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np

## Download the data from Pangaea

In [ ]:
files = {'Moulin_L41A_ice_ablation.tab': 'https://doi.pangaea.de/10.1594/PANGAEA.926842?format=textfile',
         'Moulin_L41A_Q.tab': 'https://doi.pangaea.de/10.1594/PANGAEA.926844?format=textfile'
        }
for f in files:
    if os.path.exists(f):
        print('File already downloaded.')
    else:
        print('Downloading...')
        response = requests.get(files[f])
        open(f, 'wb').write(response.content)

## Open the datasets using Pandas

In [ ]:
ablation = pd.read_csv('Moulin_L41A_ice_ablation.tab', sep=r'\t', skiprows=19, parse_dates=True, index_col=0, engine='python')

In [ ]:
discharge = pd.read_csv('Moulin_L41A_Q.tab', sep=r'\t', skiprows=20, parse_dates=True, index_col=0, engine='python')

## Initial look at dataset structure

In [ ]:
# In Jupyter Notebook, entering the name of a Pandas DataFrame prints its head and foot to screen
ablation

In [ ]:
# We can also get some basic statistics calculated over the whole DataFrame
ablation.describe()

In [ ]:
discharge

Notice that there are three columns of data in the discharge dataset. The columns with 'min' and 'max' in the titles are actually the - and + error bounds corresponding to the 95% confidence intervals. They are *not* the absolute min and max values. See the plotting code below for an example of how to use these columns.

## Plot the data

In [ ]:
# Add the ablation data above, i.e. do a subplots plot
plt.figure()
ax_ablat = plt.subplot(211)
ax_disch = plt.subplot(212, sharex=ax_ablat)

ax_ablat.errorbar(ablation.index+pd.Timedelta(hours=12), ablation['Ablation [mm]'], yerr=ablation['Ablation [±]'], 
                  drawstyle='steps-mid', elinewidth=0.7, ecolor='gray')
ax_ablat.grid()
ax_ablat.set_ylabel('Ablation (mm)')

# Discharge at the confidence bounds
lower_q = discharge['Q hour mean [m**3/s]'] + discharge['Q hour max [m**3/s]']
upper_q = discharge['Q hour mean [m**3/s]'] - discharge['Q hour min [m**3/s]']

# Plot as a filled area
ax_disch.fill_between(lower_q.index, lower_q, upper_q, alpha=0.5)
discharge['Q hour mean [m**3/s]'].plot(ax=ax_disch)
ax_disch.grid()
ax_disch.set_ylabel(r'Q ($m^3s^{-1}$)')

# Plot vertical lines
#ax_disch.axvline('2012-07-08', color='red')

# Or plot shaded areas
#ax_disch.axvspan('2012-07-09', '2012-07-11', alpha=0.3, color='red')

## Questions / Exercises

### Based only on what you see in these data, do you think that you are looking at snow melt or ice melt?

### How much water flows into the moulin per day, expressed as Olympic swimming pools?

In [ ]:
# Resample the discharge data to daily sums, matching sampling of ablation data
# Note the min_count is set to the number of hours in a day, so that we only
# look at days with full temporal data
# We multiply by 60secs * 60 mins to convert m3s-1 into m3day-1.
q_daily = discharge['Q hour mean [m**3/s]'].resample('1D').sum(min_count=24) * 60 * 60

In [ ]:
vol_pool_m3 = # some value, go find it
# now calculate the mean daily discharge and divide it by this volume.

### Is there a significant correlation between daily ablation and daily discharge?

*To answer this question you will find the code below helpful.*

In [ ]:
import statsmodels.api as sm
# Do an 'Ordinary Least Squares' regression
# See https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html for more information

# Cut ablation data to same time period as discharge (discharge is shorter)
X = ablation.loc[q_daily.index[0]:q_daily.index[-1]]['Ablation [mm]']

# Get the discharge mean series
Y = q_daily

# Some days are missing data; we 'drop' (remove) those days.
model = sm.OLS(Y, sm.add_constant(X), missing='drop')

fit = model.fit()
fit.summary()

Put your answer to this question below.

### Is this value what you expected, and why?

### When is peak daily discharge?

### When is base (minimum) daily discharge?